In [1]:
import warnings
warnings.filterwarnings('ignore')

#from src.local_models.llm import MyLLM, LlamaCPPLLM
from llama_index.llms.openai import OpenAI
from src.local_models.embeddings import get_embed_model
from llama_index.core import ServiceContext

from src.data_loader.splitting import split_by_md_headers, text_2_Document
from src.data_loader.chunking import chunk_docs_standalone
from src.data_loader.load_from_dir import rebuild_index

#from llama_index.core import Document
import os, re
import pandas as pd

from typing import Dict, List
from dotenv import load_dotenv

In [14]:
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.core import PromptTemplate
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.schema import IndexNode

from llama_index.agent.openai import OpenAIAgent
#from llama_index.core.agent import ReActAgent

from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from src.utils import load_prompt

In [3]:
load_dotenv(override=True)

True

## LLM and Embeddings

In [4]:
embed_model = get_embed_model(model_name=os.environ['embed_path'],  model_kwargs={'device': 'cpu'}, encode_kwargs = {'normalize_embeddings': True})
#llm = MyLLM(pretrained_model_name_or_path=os.environ['llm_path'], device_map="mps", context_window=4096, num_output=512, model_name='chatglm3-6b')
llm = OpenAI()
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)


## Load data as DF and apply chunking to each content cell

In [5]:
class MDDF:
    def __init__(self, df: pd.DataFrame, remove_rows: List=None):
        self.df = df.fillna('')
        if remove_rows:
            self.df.drop(remove_rows, inplace=True)
            self.df.reset_index(drop=True, inplace=True)
        #self.key_words, self.docs = self.construct_node_mappings()
        


    @staticmethod
    def remove_section_number(text):
        pattern = r'^\d+(\.\d+)*\s+'  # Matches section number followed by whitespace
        return re.sub(pattern, '', text)
    @property
    def get_shape(self):
        return self.df.shape
    
    
    def construct_node_mappings(self, chunk_size=1024, chunk_overlap=20, show_progress=True):
        ncol = self.get_shape[1]
        self.df[self.df.columns[:ncol-1]] = self.df[self.df.columns[:ncol-1]].map(MDDF.remove_section_number)
        

        self.df['key_words'] = self.df.iloc[:, :ncol-1].apply(
            lambda row: '文档标题、创建时间、标签和类别' if row.isna().all() or all(val == '' for val in row) else ', '.join(filter(bool, row.dropna())).strip(),
            axis=1
        )
        
        docs = {}
        key_words = []
        
        for idx, row in self.df.iterrows():
            kw = row['key_words']
            key_words.append(kw)
            docs[kw] = chunk_docs_standalone(
                documents=text_2_Document(strr=row['content'], meta={}),
                chunk_size=chunk_size, chunk_overlap=chunk_overlap, show_progress=show_progress)
        
        return key_words, docs

    
    
    @property
    def show_df(self):
        return self.df
    @property
    def show_mappings(self):
        return self.docs



In [6]:
#format md as DF
df = split_by_md_headers('data/产品白皮书/KDP-WhitePaper.md')
#construct node mappings
key_words, docs = MDDF(df, [1]).construct_node_mappings(show_progress=False)#remove useless contents

### Build Query Engine (Document Agents)

In [15]:
#load prompt
prompt_str = load_prompt('prompt_bank/whitepaper.txt')
new_vector_tmpl = PromptTemplate(prompt_str)

In [17]:
def build_doc_agent_engine(key_words, docs=None, similarity_top_k=None):
    ##Build Document Agent for each Document
    
    #build agents dict
    agents = {}
    nodes = []
    for i, kw in enumerate(key_words):

        #build vector index--first time
        #vector_index = VectorStoreIndex(docs[kw], embed_model=embed_model)
        #vector_index.storage_context.persist(persist_dir="db_stores/doc_agent_vector_index")

        #load from disk
        vector_index = rebuild_index(persist_dir=f'db_stores/doc_agent_vector_index/idx_{i}', service_context=service_context)


        #build keyword indexfirst time
        #kw_index = KeywordTableIndex.from_docunments(docs[kw])
        #summary_index = SummaryIndex(docs[kw], embed_model=embed_model)
        #summary_index.storage_context.persist(persist_dir="db_stores/doc_agent_summary_index")

        #load from disk
        summary_index = rebuild_index(persist_dir=f'db_stores/doc_agent_summary_index/idx_{i}', service_context=service_context)



        #define query engines
        vector_query_engine = vector_index.as_query_engine(llm=llm)
        vector_query_engine.update_prompts({'response_synthesizer:text_qa_template': new_vector_tmpl})
        
        #kw_query_engine = kw_index.as_query_engine()
        list_query_engine = summary_index.as_query_engine(llm=llm)
        list_query_engine.update_prompts({'response_synthesizer:text_qa_template': new_vector_tmpl})

        #define tools
        query_engine_tools = [
            QueryEngineTool(
                query_engine=vector_query_engine,
                metadata=ToolMetadata(
                    name="vector_tool",
                    description=(
                        f"Useful for retrieving specific context from {kw}"
                    )
                )
            ),
            QueryEngineTool(
                #query_engine=kw_query_engine,
                query_engine=list_query_engine,
                metadata=ToolMetadata(
                    name="summary_tool",
                    description=(
                        f"Useful for summarization-wise questions related to {kw}"
                    )
                )
            )
        ]
        

        #build agents
        agent = OpenAIAgent.from_tools(
            query_engine_tools,
            llm=llm,
            embed_model=embed_model,
            verbose=True,
            #output_parser=output_parser, 
            
            system_prompt=f"""\
                Make sure to respond in Chinese.

                You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.

                Please refer to the summary_tool first if you inquire general-purpose questions about the {kw},
                for which a summary suffices.

                For questions involving specific facts, such as step-by-step instructions or troubleshooting issues, 
                please refer to the vector_tool first, as such questions demand a
                finer degree of information granularity.
                """,
        )#ReActAgent
        agents[kw] = agent
    
    
        ##Build Composable Retriever over the agents
        #define top-level nodes
        instru =(
            f"{kw} are the headers and subheaders corresponding to specific paragraphs from the KDP whitepaper, "
            "(To clarify, KDP stands for Kubernetes Data Platform, "
            "which is a enterprise-level big data platform developed by Linktime Cloud Co., Ltd.) "
            "You are a well-versed agent designed to answer queries about the following aspect of the KDP whitepaper, "
            f"{kw} "
            "and you know by heart the core concepts and mechanisms of big data technologies. "
            f"This paragraph contains contents associated with the topic {kw}. "
            "Use this index if you need to look up information about {kw}, "
            "Do not use this index if you want to gather information from multiple aspects beyond {kw}"
        )

        node = IndexNode(
            text=instru, index_id=kw, obj= agent
            )
        nodes.append(node)
    
    #define top-level retriever
    top_vector_index = VectorStoreIndex(objects=nodes, embed_model=embed_model)
    query_engine = top_vector_index.as_query_engine(similarity_top_k=similarity_top_k, verbose=True)
    return query_engine


In [18]:
t = build_doc_agent_engine(key_words, similarity_top_k=2)

In [19]:
t.query("KDP的应用场景有哪些")

Retrieval entering 应用场景: OpenAIAgent
Retrieving from object OpenAIAgent with query KDP的应用场景有哪些
Added user message to memory: KDP的应用场景有哪些
=== Calling Function ===
Calling function: summary_tool with args: {"input":"KDP的应用场景"}
Got output: KDP的应用场景主要包括但不限于以下几个方面：
1. 在多租户环境下进行大数据开发，提供高效、安全、稳定的大数据计算及存储引擎。
2. 取代传统大数据平台，帮助企业在数字化转型过程中实现降本增效的目标。
3. 实现大数据组件的部署、配置和运维，是大数据技术落地实施的最佳技术选择。
4. 深度整合了云原生架构的优势，将大数据组件及数据应用纳入Kubernetes管理体系，实现更高效的运行和协同工作。

Retrieval entering 应用场景, 提升IT架构资源效率: OpenAIAgent
Retrieving from object OpenAIAgent with query KDP的应用场景有哪些
Added user message to memory: KDP的应用场景有哪些
=== Calling Function ===
Calling function: summary_tool with args: {"input":"KDP的应用场景"}
Got output: KDP的应用场景包括但不限于以下几个方面：
1. 在生产环境中运行多种类型的数据应用、不同类型的存储引擎、实时和批处理的计算作业。
2. 提升整体资源使用效率，降低IT架构的投入成本。
3. 实现作业混排、存算分离和精细化调度，以提高资源利用率。
4. 适用于需要在Kubernetes上部署容器化云原生大数据平台的企业。



Response(response='KDP的应用场景主要包括在多租户环境下进行大数据开发、取代传统大数据平台、实现大数据组件的部署和运维，以及深度整合云原生架构的优势等方面。具体包括在生产环境中运行多种类型的数据应用、不同类型的存储引擎、实时和批处理的计算作业；提升整体资源使用效率，降低IT架构的投入成本；实现作业混排、存算分离和精细化调度，以提高资源利用率；适用于需要在Kubernetes上部署容器化云原生大数据平台的企业。', source_nodes=[NodeWithScore(node=TextNode(id_='3cd9b31b-d1b1-43b2-8669-d1a051cd569a', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='KDP的应用场景主要包括在多租户环境下进行大数据开发、取代传统大数据平台、实现大数据组件的部署和运维，以及深度整合云原生架构的优势等方面。', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.821623414440564), NodeWithScore(node=TextNode(id_='135df706-ffd5-4f38-992c-e522a1aa9c9f', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='KDP的应用场景包括但不限于以下几个方面：\n1. 在生产环境中运行多种类型的数据应用、不同类型的存储引擎、实时和批处理的计算作业。\n2. 提升整体资源使用效率，降低IT架构的投入成本。\n3. 实现作业混排、存算分离和精细化调度，以提高资源利用率。\n4. 适用于需要在Kuberne

In [11]:
t.query("KDP的优势是什么")

Retrieval entering KDP对大数据平台的云原生改造, 组件K8s性能优化: OpenAIAgent
Retrieving from object OpenAIAgent with query KDP的优势是什么
Added user message to memory: KDP的优势是什么
=== Calling Function ===
Calling function: summary_tool with args: {"input":"KDP的优势"}
Got output: KDP的优势包括对大数据组件在K8s上的性能优化，解决了云原生环境下的data locality问题和通过持续运行机制避免了Spark pod的频繁启动。这些优化措施有助于提升大数据组件在K8s上的运行效率，减少性能损耗，以及更好地利用云原生的机制来处理各种特性的大数据工作负载。



Response(response='KDP的优势包括对大数据组件在K8s上的性能优化，解决了云原生环境下的data locality问题和通过持续运行机制避免了Spark pod的频繁启动。这些优化措施有助于提升大数据组件在K8s上的运行效率，减少性能损耗，以及更好地利用云原生的机制来处理各种特性的大数据工作负载。', source_nodes=[NodeWithScore(node=TextNode(id_='5a6b7dcb-14cd-4475-80fd-ce591cf744b7', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='KDP的优势包括对大数据组件在K8s上的性能优化，解决了云原生环境下的data locality问题和通过持续运行机制避免了Spark pod的频繁启动。这些优化措施有助于提升大数据组件在K8s上的运行效率，减少性能损耗，以及更好地利用云原生的机制来处理各种特性的大数据工作负载。', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.7921375744187213)], metadata={'5a6b7dcb-14cd-4475-80fd-ce591cf744b7': {}})

In [9]:
t.query("KDP集成的kafka版本是多少")

Retrieval entering KDP系统架构及功能组件, 计算引擎云原生改造及集成, Kafka: OpenAIAgent
Retrieving from object OpenAIAgent with query KDP集成的kafka版本是多少
Added user message to memory: KDP集成的kafka版本是多少
=== Calling Function ===
Calling function: vector_tool with args: {"input":"KDP集成的kafka版本是多少"}
Got output: KDP集成的Kafka版本是2.7.0。



Response(response='KDP集成的Kafka版本是2.7.0。', source_nodes=[NodeWithScore(node=TextNode(id_='35667742-0cd7-4222-88a7-9e54efd5e0fc', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='KDP集成的Kafka版本是2.7.0。', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.8189683110618228)], metadata={'35667742-0cd7-4222-88a7-9e54efd5e0fc': {}})

In [10]:
t.query('大数据组件如何部署')

Retrieval entering 云原生大数据架构优势, 简化大数据组件集成及配置安装流程: OpenAIAgent
Retrieving from object OpenAIAgent with query 大数据组件如何部署
Added user message to memory: 大数据组件如何部署
=== Calling Function ===
Calling function: vector_tool with args: {"input":"大数据组件如何部署"}
Got output: 大数据组件可以通过智领云研发团队自主研发的KDP大数据集成基座来部署。该集成基座采用了统一的K8s文件配置方式对大数据组件的镜像文件、软件依赖、部署步骤、与K8s网络和存储的对接、计算和存储资源的配额、监控指标的输出、日志文件的采集等内容进行描述。安装程序会通过这些配置文件自动执行大数据组件到K8s集群的部署、更新、运维、升级等操作，从而简化了大数据组件与K8s集群的集成，降低了大数据平台部署的复杂性，并减少了大数据平台运维的成本。



Response(response='大数据组件可以通过智领云研发团队自主研发的KDP大数据集成基座来部署。集成基座使用统一的K8s文件配置方式描述大数据组件的镜像文件、软件依赖、部署步骤、与K8s网络和存储的对接、计算和存储资源的配额、监控指标的输出、日志文件的采集等内容。安装程序会根据这些配置文件自动执行大数据组件到K8s集群的部署、更新、运维、升级等操作，简化了大数据组件与K8s集群的集成，降低了大数据平台部署的复杂性，并减少了大数据平台运维的成本。', source_nodes=[NodeWithScore(node=TextNode(id_='4c752c66-b939-434f-96e4-94c3b51d4c97', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='大数据组件可以通过智领云研发团队自主研发的KDP大数据集成基座来部署。该集成基座采用了统一的K8s文件配置方式对大数据组件的镜像文件、软件依赖、部署步骤、与K8s网络和存储的对接、计算和存储资源的配额、监控指标的输出、日志文件的采集等内容进行描述。安装程序会通过这些配置文件自动执行大数据组件到K8s集群的部署、更新、运维、升级等操作，从而简化了大数据组件与K8s集群的集成，降低了大数据平台部署的复杂性，并减少了大数据平台运维的成本。', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.8243962823787884)], metadata={'4c752c66-b939-434f-96e4-94c3b51d4c97': {}})

In [ ]:
#response_modes = ['refine', 'compact', 'tree_summarize', 'simple_summarize', 'no_text', 'accumulate', 'compact_accumulate']
